BASIC - LR

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("LinReg").getOrCreate()

In [2]:
from pyspark.ml.regression import LinearRegression

In [3]:
training = spark.read.format("libsvm").load("sample_linear_regression_data.txt")

In [5]:
training.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
| -9.490009878824548|(10,[0,1,2,3,4,5,...|
| 0.2577820163584905|(10,[0,1,2,3,4,5,...|
| -4.438869807456516|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
| -7.966593841555266|(10,[0,1,2,3,4,5,...|
| -7.896274316726144|(10,[0,1,2,3,4,5,...|
| -8.464803554195287|(10,[0,1,2,3,4,5,...|
| 2.1214592666251364|(10,[0,1,2,3,4,5,...|
| 1.0720117616524107|(10,[0,1,2,3,4,5,...|
|-13.772441561702871|(10,[0,1,2,3,4,5,...|
| -5.082010756207233|(10,[0,1,2,3,4,5,...|
|  7.887786536531237|(10,[0,1,2,3,4,5,...|
| 14.323146365332388|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
|-0.8995693247765151|(10,[0,1,2,3,4,5,...|
| -19.16829262296376|(10,[0,1,2,3,4,5,...|
|  5.601801561245534|(10,[0,1,2,3,4,5,...|
|-3.2256352187273354|(10,[0,1,2,3,4,5,...|
| 1.5299675726687754|(10,[0,1,2,3,4,5,...|
| -0.250102447941961|(10,[0,1,2,3,4,5,...|
+----------

In [6]:
lr = LinearRegression(featuresCol="features", labelCol="label", predictionCol="prediction")

In [7]:
lrModel = lr.fit(training)

In [9]:
lrModel.coefficients

DenseVector([0.0073, 0.8314, -0.8095, 2.4412, 0.5192, 1.1535, -0.2989, -0.5129, -0.6197, 0.6956])

In [10]:
lrModel.intercept

0.14228558260358093

In [12]:
training_summary = lrModel.summary

In [13]:
training_summary.r2

0.027839179518600154

In [15]:
training_summary.rootMeanSquaredError

10.16309157133015

In [16]:
all_data = spark.read.format("libsvm").load("sample_linear_regression_data.txt")

In [22]:
train_data, test_data = all_data.randomSplit([0.75,0.25])

In [23]:
train_data.count()

371

In [24]:
test_data.count()

130

In [25]:
correct_model = lr.fit(train_data)

In [26]:
test_results = correct_model.evaluate(test_data)

In [28]:
test_results.rootMeanSquaredError

10.829423720099882

In [29]:
unlabeled_data = test_data.select("features")

In [31]:
predictions = correct_model.transform(unlabeled_data)

In [33]:
predictions.show()

+--------------------+--------------------+
|            features|          prediction|
+--------------------+--------------------+
|(10,[0,1,2,3,4,5,...|  -4.423816645859675|
|(10,[0,1,2,3,4,5,...| -0.6371319709324597|
|(10,[0,1,2,3,4,5,...|   4.009576451444203|
|(10,[0,1,2,3,4,5,...| 0.13663928504330924|
|(10,[0,1,2,3,4,5,...|  0.9293057846512529|
|(10,[0,1,2,3,4,5,...|-0.18752673934369457|
|(10,[0,1,2,3,4,5,...| -1.7167277985780574|
|(10,[0,1,2,3,4,5,...| -0.6729839957008452|
|(10,[0,1,2,3,4,5,...|-0.06824101572590502|
|(10,[0,1,2,3,4,5,...|  1.8455789578565909|
|(10,[0,1,2,3,4,5,...|  0.6621014050022591|
|(10,[0,1,2,3,4,5,...|   2.262178152652472|
|(10,[0,1,2,3,4,5,...| -1.4711719208559508|
|(10,[0,1,2,3,4,5,...|   3.190814941541559|
|(10,[0,1,2,3,4,5,...|  -1.153547303423968|
|(10,[0,1,2,3,4,5,...|  1.1449430405938328|
|(10,[0,1,2,3,4,5,...| -1.6229132436169567|
|(10,[0,1,2,3,4,5,...|   2.188590099846662|
|(10,[0,1,2,3,4,5,...|  0.6538177412820722|
|(10,[0,1,2,3,4,5,...| -1.397551

LR - Example

In [34]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('lr_example').getOrCreate()

In [35]:
from pyspark.ml.regression import LinearRegression

In [37]:
data = spark.read.csv("Ecommerce_Customers.csv", inferSchema=True, header=True)

In [39]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [46]:
for item in data.head():
    print(item)

mstephenson@fernandez.com
835 Frank TunnelWrightmouth, MI 82180-9605
Violet
34.49726772511229
12.65565114916675
39.57766801952616
4.0826206329529615
587.9510539684005


## Setting Up DataFrame for Machine Learning 

In [47]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [48]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [50]:
assembler = VectorAssembler(inputCols=['Avg Session Length', 
                                       'Time on App',
                                       'Time on Website',
                                       'Length of Membership'], 
                            outputCol="features")

In [51]:
output = assembler.transform(data)

In [52]:
output

DataFrame[Email: string, Address: string, Avatar: string, Avg Session Length: double, Time on App: double, Time on Website: double, Length of Membership: double, Yearly Amount Spent: double, features: vector]

In [53]:
output.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)
 |-- features: vector (nullable = true)



In [55]:
output.select("features").head()

Row(features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]))

In [56]:
final_data = output.select("features", "Yearly Amount Spent")

In [57]:
train_data, test_data = final_data.randomSplit([0.7,0.3])

In [59]:
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                356|
|   mean| 500.09907741837264|
| stddev|  79.12110680983898|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [60]:
test_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                144|
|   mean|   497.373247002464|
| stddev|  80.03546265132702|
|    min|  282.4712457199145|
|    max|  725.5848140556806|
+-------+-------------------+



In [61]:
lr = LinearRegression(labelCol="Yearly Amount Spent")

In [62]:
lrModel = lr.fit(train_data)

In [63]:
test_results = lrModel.evaluate(test_data)

In [68]:
test_results.residuals.show()

+--------------------+
|           residuals|
+--------------------+
|  -4.634718697179153|
|  0.2238594005279424|
|  11.259411256658609|
|-0.24395085163280328|
|  3.3629643701175382|
| 0.25736931435062615|
|  -5.530839047835457|
|  -9.157037204594815|
|  3.8175929320100863|
|   17.67617515955436|
|  -5.529620451346943|
| -11.041771545382062|
|   7.286236273780503|
|  -9.138114517074825|
| -2.2944557133002945|
|  -6.037413018356517|
| -1.5681792042097982|
|  11.775981717734624|
|  5.6474067984235035|
|  -4.906343364232839|
+--------------------+
only showing top 20 rows



In [64]:
test_results.rootMeanSquaredError

9.424014066154538

In [71]:
test_results.r2 #model explains 98% variance in the data

0.9860384580221198

In [72]:
test_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                144|
|   mean|   497.373247002464|
| stddev|  80.03546265132702|
|    min|  282.4712457199145|
|    max|  725.5848140556806|
+-------+-------------------+



In [73]:
unlabeled_data = test_data.select("features")

In [75]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|[30.4925366965402...|
|[30.5743636841713...|
|[30.7377203726281...|
|[31.2606468698795...|
|[31.3091926408918...|
|[31.3895854806643...|
|[31.5257524169682...|
|[31.5261978982398...|
|[31.5316044825729...|
|[31.6098395733896...|
|[31.7242025238451...|
|[31.8093003166791...|
|[31.8209982016720...|
|[31.8279790554652...|
|[31.8530748017465...|
|[31.8745516945853...|
|[31.9120759292006...|
|[31.9262720263601...|
|[31.9480174211613...|
|[31.9673209478824...|
+--------------------+
only showing top 20 rows



In [78]:
predicted_data = lrModel.transform(unlabeled_data).show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[30.4925366965402...| 287.1059644170937|
|[30.5743636841713...| 441.8405543575377|
|[30.7377203726281...|450.52133093957127|
|[31.2606468698795...| 421.5705821085842|
|[31.3091926408918...| 429.3577534698161|
|[31.3895854806643...|409.81224174563226|
|[31.5257524169682...|449.49646585771734|
|[31.5261978982398...| 418.2515633969326|
|[31.5316044825729...| 432.6980127973525|
|[31.6098395733896...| 426.8693744915538|
|[31.7242025238451...|508.91750773930744|
|[31.8093003166791...| 547.8136709082232|
|[31.8209982016720...|417.38904473943285|
|[31.8279790554652...|449.14086206401635|
|[31.8530748017465...|461.57957917565227|
|[31.8745516945853...|  398.322657264624|
|[31.9120759292006...| 389.1028955099175|
|[31.9262720263601...| 380.4289517265918|
|[31.9480174211613...| 456.2734700944743|
|[31.9673209478824...| 450.6561846038851|
+--------------------+------------